In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_predict
from numpy import mean
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from matplotlib.colors import Normalize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_original = pd.read_csv('/content/drive/MyDrive/Prospect Summary.txt',delimiter='\t', header=0)

In [4]:
df_original['ID'] = range(1,len(df_original)+1)

In [5]:
df_original

,RegardingObjectId,Booking date,Phone Call Interval,Phone Call before booking,isbooking,Total Booking,isinLead,created Year,Leadindex,Lead Margin,Payment Type,Visit before booking,Test Drive before booking,Last Phone Call Date before booking,Last Phone Call Group,Month Year,Month Year Value,First Phone Call Date,hourdiff_from_created,created date,booking_first phone call_datediff,booking_created_date_datediff,Booking Index,First Phone Call Index,isBookingbehindPhonecall,Month,Lead Channel ID,Channel Name,ID
0,AEC7157D,NaN,1.0,551.0,No Booking,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,12.0,Middle Month,NaN,NaN,4/10/2019 3:52:01 AM,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,1
1,CBDF780C,NaN,1.0,430.0,No Booking,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,16.0,Middle Month,NaN,NaN,1/13/2020 8:39:32 AM,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,2
2,9F28BD74,NaN,1.0,244.0,No Booking,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Early Month,NaN,NaN,11/27/2019 6:49:24 AM,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,3
3,43A9C7FA,NaN,1.0,100.0,No Booking,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,25.0,Lately Month,NaN,NaN,9/25/2019 2:27:15 AM,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,4
4,5D08A6C7,NaN,1.0,93.0,No Booking,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Lately Month,NaN,NaN,11/26/2019 3:14:09 AM,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273723,93761E29,NaN,NaN,NaN,No Booking,NaN,True,2018.0,NaN,NaN,NaN,NaN,1.0,NaN,(Blank),NaN,NaN,NaN,NaN,3/3/2018 4:48:57 AM,NaN,NaN,NaN,NaN,False,3.0,100000000.0,Walk in,273724
273724,98A7B74D,NaN,NaN,NaN,No Booking,NaN,True,2018.0,NaN,NaN,NaN,NaN,1.0,NaN,(Blank),NaN,NaN,NaN,NaN,4/13/2018 4:47:08 PM,NaN,NaN,NaN,NaN,False,4.0,100000000.0,Walk in,273725
273725,9A780FDA,NaN,63.0,NaN,No Booking,NaN,True,2018.0,NaN,NaN,NaN,NaN,2.0,NaN,(Blank),NaN,NaN,NaN,NaN,4/22/2018 11:18:16 AM,NaN,NaN,NaN,NaN,False,4.0,100000000.0,Walk in,273726
273726,A5114BC8,NaN,NaN,NaN,No Booking,NaN,True,2018.0,NaN,NaN,NaN,NaN,1.0,NaN,(Blank),NaN,NaN,NaN,NaN,5/11/2018 10:19:27 AM,NaN,NaN,NaN,NaN,False,5.0,100000007.0,Internet,273727


In [6]:
df_original = df_original.fillna(0)
df_original = df_original[(df_original['isinLead']==True)&(df_original['Phone Call before booking']>0)&(df_original['Phone Call Interval']<365)&(df_original['Lead Channel ID']>0)&(df_original['hourdiff_from_created']<=720)]
drop_columns = ['RegardingObjectId','Booking date','Total Booking','isinLead','created Year','Leadindex','Lead Margin','Payment Type','Last Phone Call Date before booking','Last Phone Call Group',
                'Month Year','Month Year Value','First Phone Call Date','created date','booking_first phone call_datediff','booking_created_date_datediff','Booking Index','First Phone Call Index',
                'isBookingbehindPhonecall','Month','Lead Channel ID']
df_original = df_original.drop(columns=drop_columns,axis=1)

In [7]:
df_original.columns.values

array(['Phone Call Interval', 'Phone Call before booking', 'isbooking',
       'Visit before booking', 'Test Drive before booking',
       'hourdiff_from_created', 'Channel Name', 'ID'], dtype=object)

In [8]:
re_order = ['ID','Phone Call Interval', 'Phone Call before booking', 'isbooking',
       'Visit before booking', 'Test Drive before booking',
       'hourdiff_from_created', 'Channel Name']
df_original = df_original[re_order]
df_original['isbooking'] = df_original['isbooking'].replace(['No Booking','Booking'],[0,1])
df_original

,ID,Phone Call Interval,Phone Call before booking,isbooking,Visit before booking,Test Drive before booking,hourdiff_from_created,Channel Name
6,7,339.0,12.0,0,0.0,0.0,14.0,Event
10,11,265.0,11.0,0,0.0,0.0,20.0,Event
12,13,30.0,6.0,1,0.0,0.0,41.0,Event
20,21,196.0,9.0,0,0.0,0.0,14.0,Event
34,35,249.0,9.0,0,0.0,0.0,43.0,Event
...,...,...,...,...,...,...,...,...
271288,271289,60.0,3.0,0,0.0,0.0,0.0,Call
271329,271330,56.0,4.0,0,0.0,0.0,0.0,Walk in
271334,271335,79.0,4.0,0,0.0,0.0,47.0,Walk in
271355,271356,216.0,6.0,0,0.0,0.0,0.0,Walk in


In [9]:
def Phone_Call_interval(columns):
    if columns <= 3 :
        return '<3'
    elif columns <=10 :
        return '<=10'
    elif columns <= 20 :
        return '<=20'
    elif columns <= 30 :
        return '<=30'
    elif columns <= 60 :
        return '<=60'
    elif columns <= 90 :
        return '<=90'
    elif columns <= 180 :
        return '<=180'
    else:
        return '>180'
      

In [10]:
def myround(x, base=12):
    return int(base * round(x/base))

In [11]:
df_original['Phone Call Interval'] = df_original['Phone Call Interval'].apply(Phone_Call_interval)


In [12]:
df_original['hourdiff_from_created'] = df_original['hourdiff_from_created'].apply(myround)

In [13]:
df_original['hourdiff_from_created'] = df_original['hourdiff_from_created'].astype(str)

In [14]:
df_original

,ID,Phone Call Interval,Phone Call before booking,isbooking,Visit before booking,Test Drive before booking,hourdiff_from_created,Channel Name
6,7,>180,12.0,0,0.0,0.0,12,Event
10,11,>180,11.0,0,0.0,0.0,24,Event
12,13,<=30,6.0,1,0.0,0.0,36,Event
20,21,>180,9.0,0,0.0,0.0,12,Event
34,35,>180,9.0,0,0.0,0.0,48,Event
...,...,...,...,...,...,...,...,...
271288,271289,<=60,3.0,0,0.0,0.0,0,Call
271329,271330,<=60,4.0,0,0.0,0.0,0,Walk in
271334,271335,<=90,4.0,0,0.0,0.0,48,Walk in
271355,271356,>180,6.0,0,0.0,0.0,0,Walk in


In [15]:
df_dummy = df_original.copy()

In [16]:
cols = [0,2,3,4,5]
df_dummy = df_dummy.drop(df_dummy.columns[cols],axis=1)


In [17]:
df_dummy = pd.get_dummies(df_dummy)

In [18]:
df_dummy

,Phone Call Interval_<3,Phone Call Interval_<=10,Phone Call Interval_<=180,Phone Call Interval_<=20,Phone Call Interval_<=30,Phone Call Interval_<=60,Phone Call Interval_<=90,Phone Call Interval_>180,hourdiff_from_created_0,hourdiff_from_created_108,hourdiff_from_created_12,hourdiff_from_created_120,hourdiff_from_created_132,hourdiff_from_created_144,hourdiff_from_created_156,hourdiff_from_created_168,hourdiff_from_created_180,hourdiff_from_created_192,hourdiff_from_created_204,hourdiff_from_created_216,hourdiff_from_created_228,hourdiff_from_created_24,hourdiff_from_created_240,hourdiff_from_created_252,hourdiff_from_created_264,hourdiff_from_created_276,hourdiff_from_created_288,hourdiff_from_created_300,hourdiff_from_created_312,hourdiff_from_created_324,hourdiff_from_created_336,hourdiff_from_created_348,hourdiff_from_created_36,hourdiff_from_created_360,hourdiff_from_created_372,hourdiff_from_created_384,hourdiff_from_created_396,hourdiff_from_created_408,hourdiff_from_created_420,hourdiff_from_created_432,hourdiff_from_created_444,hourdiff_from_created_456,hourdiff_from_created_468,hourdiff_from_created_48,hourdiff_from_created_480,hourdiff_from_created_492,hourdiff_from_created_504,hourdiff_from_created_516,hourdiff_from_created_528,hourdiff_from_created_540,hourdiff_from_created_552,hourdiff_from_created_564,hourdiff_from_created_576,hourdiff_from_created_588,hourdiff_from_created_60,hourdiff_from_created_600,hourdiff_from_created_612,hourdiff_from_created_624,hourdiff_from_created_636,hourdiff_from_created_648,hourdiff_from_created_660,hourdiff_from_created_672,hourdiff_from_created_684,hourdiff_from_created_696,hourdiff_from_created_708,hourdiff_from_created_72,hourdiff_from_created_720,hourdiff_from_created_84,hourdiff_from_created_96,Channel Name_Call,Channel Name_Digital IMX,Channel Name_Event,Channel Name_Internet,Channel Name_Other,Channel Name_Referral,Channel Name_Retention,Channel Name_SEM CRM,Channel Name_Walk in
6,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
12,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
20,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271288,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
271329,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271355,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [19]:
df_original_final = pd.concat([df_original,df_dummy],axis=1)

In [20]:
df_original_final = df_original_final.drop(['Phone Call Interval','ID','Channel Name','hourdiff_from_created'],axis=1)

In [21]:
df_original_final = df_original_final.fillna(0)
df_original_final

,Phone Call before booking,isbooking,Visit before booking,Test Drive before booking,Phone Call Interval_<3,Phone Call Interval_<=10,Phone Call Interval_<=180,Phone Call Interval_<=20,Phone Call Interval_<=30,Phone Call Interval_<=60,Phone Call Interval_<=90,Phone Call Interval_>180,hourdiff_from_created_0,hourdiff_from_created_108,hourdiff_from_created_12,hourdiff_from_created_120,hourdiff_from_created_132,hourdiff_from_created_144,hourdiff_from_created_156,hourdiff_from_created_168,hourdiff_from_created_180,hourdiff_from_created_192,hourdiff_from_created_204,hourdiff_from_created_216,hourdiff_from_created_228,hourdiff_from_created_24,hourdiff_from_created_240,hourdiff_from_created_252,hourdiff_from_created_264,hourdiff_from_created_276,hourdiff_from_created_288,hourdiff_from_created_300,hourdiff_from_created_312,hourdiff_from_created_324,hourdiff_from_created_336,hourdiff_from_created_348,hourdiff_from_created_36,hourdiff_from_created_360,hourdiff_from_created_372,hourdiff_from_created_384,...,hourdiff_from_created_420,hourdiff_from_created_432,hourdiff_from_created_444,hourdiff_from_created_456,hourdiff_from_created_468,hourdiff_from_created_48,hourdiff_from_created_480,hourdiff_from_created_492,hourdiff_from_created_504,hourdiff_from_created_516,hourdiff_from_created_528,hourdiff_from_created_540,hourdiff_from_created_552,hourdiff_from_created_564,hourdiff_from_created_576,hourdiff_from_created_588,hourdiff_from_created_60,hourdiff_from_created_600,hourdiff_from_created_612,hourdiff_from_created_624,hourdiff_from_created_636,hourdiff_from_created_648,hourdiff_from_created_660,hourdiff_from_created_672,hourdiff_from_created_684,hourdiff_from_created_696,hourdiff_from_created_708,hourdiff_from_created_72,hourdiff_from_created_720,hourdiff_from_created_84,hourdiff_from_created_96,Channel Name_Call,Channel Name_Digital IMX,Channel Name_Event,Channel Name_Internet,Channel Name_Other,Channel Name_Referral,Channel Name_Retention,Channel Name_SEM CRM,Channel Name_Walk in
6,12.0,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,11.0,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
12,6.0,1,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
20,9.0,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34,9.0,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271288,3.0,0,0.0,0.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
271329,4.0,0,0.0,0.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271334,4.0,0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271355,6.0,0,0.0,0.0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [22]:
df_final = df_original_final.copy()


In [23]:
df_final_scaled = df_final.copy()

col_names = ['Phone Call before booking','Visit before booking','Test Drive before booking']
features = df_final_scaled[col_names]

# Use scaler of choice; here Standard scaler is used
scaler = StandardScaler().fit(features.values)
features = scaler.fit_transform(features.values)

df_final_scaled[col_names] = features

In [24]:
df_final_scaled

,Phone Call before booking,isbooking,Visit before booking,Test Drive before booking,Phone Call Interval_<3,Phone Call Interval_<=10,Phone Call Interval_<=180,Phone Call Interval_<=20,Phone Call Interval_<=30,Phone Call Interval_<=60,Phone Call Interval_<=90,Phone Call Interval_>180,hourdiff_from_created_0,hourdiff_from_created_108,hourdiff_from_created_12,hourdiff_from_created_120,hourdiff_from_created_132,hourdiff_from_created_144,hourdiff_from_created_156,hourdiff_from_created_168,hourdiff_from_created_180,hourdiff_from_created_192,hourdiff_from_created_204,hourdiff_from_created_216,hourdiff_from_created_228,hourdiff_from_created_24,hourdiff_from_created_240,hourdiff_from_created_252,hourdiff_from_created_264,hourdiff_from_created_276,hourdiff_from_created_288,hourdiff_from_created_300,hourdiff_from_created_312,hourdiff_from_created_324,hourdiff_from_created_336,hourdiff_from_created_348,hourdiff_from_created_36,hourdiff_from_created_360,hourdiff_from_created_372,hourdiff_from_created_384,...,hourdiff_from_created_420,hourdiff_from_created_432,hourdiff_from_created_444,hourdiff_from_created_456,hourdiff_from_created_468,hourdiff_from_created_48,hourdiff_from_created_480,hourdiff_from_created_492,hourdiff_from_created_504,hourdiff_from_created_516,hourdiff_from_created_528,hourdiff_from_created_540,hourdiff_from_created_552,hourdiff_from_created_564,hourdiff_from_created_576,hourdiff_from_created_588,hourdiff_from_created_60,hourdiff_from_created_600,hourdiff_from_created_612,hourdiff_from_created_624,hourdiff_from_created_636,hourdiff_from_created_648,hourdiff_from_created_660,hourdiff_from_created_672,hourdiff_from_created_684,hourdiff_from_created_696,hourdiff_from_created_708,hourdiff_from_created_72,hourdiff_from_created_720,hourdiff_from_created_84,hourdiff_from_created_96,Channel Name_Call,Channel Name_Digital IMX,Channel Name_Event,Channel Name_Internet,Channel Name_Other,Channel Name_Referral,Channel Name_Retention,Channel Name_SEM CRM,Channel Name_Walk in
6,0.542736,0,-0.210423,-0.286079,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,0.449339,0,-0.210423,-0.286079,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
12,-0.017645,1,-0.210423,-0.286079,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
20,0.262546,0,-0.210423,-0.286079,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34,0.262546,0,-0.210423,-0.286079,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271288,-0.297835,0,-0.210423,-0.286079,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
271329,-0.204438,0,-0.210423,-0.286079,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271334,-0.204438,0,-0.210423,-0.286079,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
271355,-0.017645,0,-0.210423,-0.286079,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0

In [25]:
df_no_booking = df_final_scaled.copy()

In [26]:
df_no_booking = df_no_booking[(df_no_booking['isbooking'])==0]
df_no_booking_for_learn = df_no_booking.iloc[:int(round(len(df_no_booking)/2,0)),:]
df_no_booking_for_predict = df_no_booking.iloc[int(round(len(df_no_booking)/2,0)):,:]

In [27]:
df_booking = df_final_scaled.copy()

In [28]:
df_booking = df_booking[(df_booking['isbooking']==1)]
df_booking_for_learn = df_booking.iloc[:int(round(len(df_booking)/2,0)),:]
df_booking_for_predict = df_booking.iloc[int(round(len(df_booking)/2,0)):,:]

In [29]:
df_final_learn = df_no_booking_for_learn.append(df_booking_for_learn)
df_final_predict = df_no_booking_for_predict.append(df_booking_for_predict)
df_final_learn = shuffle(df_final_learn,random_state=999)
df_final_learn_predict = shuffle(df_final_predict,random_state=999)

In [30]:
X = df_final_learn.drop('isbooking',axis=1)
y = df_final_learn['isbooking']
X_for_predict = df_final_predict.drop('isbooking',axis=1)
y_for_predict = df_final_predict['isbooking']

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state = 1000)

In [32]:
svclassifier = SVC(kernel='rbf',class_weight='balanced')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [33]:
le = LogisticRegression(class_weight={0:0.26,1:0.74}, random_state=10000)
le.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight={0: 0.26, 1: 0.74}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=10000, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred = svclassifier.predict(X_test)
y_pred_le = le.predict(X_test)
print(pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_test, y_pred))
print(pd.crosstab(y_test, y_pred_le, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_test, y_pred_le))

Predicted     0     1   All
Actual                     
0          4664   571  5235
1            98   549   647
All        4762  1120  5882
              precision    recall  f1-score   support

           0       0.98      0.89      0.93      5235
           1       0.49      0.85      0.62       647

    accuracy                           0.89      5882
   macro avg       0.73      0.87      0.78      5882
weighted avg       0.93      0.89      0.90      5882

Predicted     0    1   All
Actual                    
0          5025  210  5235
1           197  450   647
All        5222  660  5882
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      5235
           1       0.68      0.70      0.69       647

    accuracy                           0.93      5882
   macro avg       0.82      0.83      0.82      5882
weighted avg       0.93      0.93      0.93      5882



In [35]:
y_pred_to_new_set = svclassifier.predict(X_for_predict)
y_pred_le_new_set = le.predict(X_for_predict)
print(pd.crosstab(y_for_predict, y_pred_to_new_set, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_for_predict, y_pred_to_new_set))
print(pd.crosstab(y_for_predict, y_pred_le_new_set, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_for_predict,  y_pred_le_new_set))

KeyboardInterrupt: ignored

In [ ]:
lr = SVC()
#Setting the range for class weights
weights = np.linspace(0.0,0.99,10)

#Creating a dictionary grid for grid search
param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}


#Fitting grid search to the train data with 5 folds
gridsearch_recall = GridSearchCV(estimator= lr, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(n_splits=3), 
                          n_jobs=-1, 
                          scoring='recall', 
                          verbose=2).fit(X_train, y_train)

gridsearch_precision = GridSearchCV(estimator= lr, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(n_splits=3), 
                          n_jobs=-1, 
                          scoring='precision', 
                          verbose=2).fit(X_train, y_train)

#Ploting the score for different values of weight


In [ ]:
test = pd.DataFrame(gridsearch_recall.cv_results_)

In [ ]:
test

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(12,8))
weight_data = pd.DataFrame({'recall': gridsearch_recall.cv_results_['mean_test_score'],'precision': gridsearch_precision.cv_results_['mean_test_score'], 'weight': (1- weights)})
sns.lineplot(weight_data['weight'],weight_data['precision'])
plt.xlabel('Weight for class 1')
plt.ylabel('score')
plt.xticks([round(i/10,1) for i in range(0,11,1)])
plt.title('Scoring for different class weights', fontsize=24)

In [ ]:

ax = weight_data.plot(x='weight', y='recall', xticks=np.arange(0,1,0.05),legend=False,figsize=(12,8))
ax2 = ax.twinx()
weight_data.plot(x='weight',y='precision', ax=ax2, legend=False, color="r")
ax.figure.legend()
ax.set_ylim(0,1)
ax2.set_ylim(0,1)

plt.show()

In [58]:
C_range = np.logspace(10, -2, 1)
gamma_range = np.logspace(-9, 3, 1)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedKFold(n_splits=2)
grid = GridSearchCV(SVC(class_weight={0:0.26,1:0.74},kernel='rbf'), param_grid=param_grid, cv=cv,n_jobs=-1,verbose=2)
grid.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   24.2s finished


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight={0: 0.26, 1: 0.74}, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e+10]), 'gamma': array([1.e-09])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [59]:
y_pred_test = grid.predict(X_test)

print(pd.crosstab(y_test, y_pred_test, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_test, y_pred_test))


Predicted     0    1   All
Actual                    
0          5196   39  5235
1           358  289   647
All        5554  328  5882
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5235
           1       0.88      0.45      0.59       647

    accuracy                           0.93      5882
   macro avg       0.91      0.72      0.78      5882
weighted avg       0.93      0.93      0.92      5882



In [56]:
scores = grid.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(gamma_range))


array([[0.93458579]])

In [ ]:
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('Validation accuracy')
plt.show()

In [61]:
svc = SVC(class_weight={0:0.26,1:0.74})
C_range = np.logspace(-2, 10, 3)

#Creating a dictionary grid for grid search
param_grid = dict(C=C_range)


#Fitting grid search to the train data with 5 folds
gridsearch_recall_C = GridSearchCV(estimator= svc, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(n_splits=2), 
                          n_jobs=-1, 
                          scoring='recall', 
                          verbose=2).fit(X_train, y_train)

gridsearch_precision_C = GridSearchCV(estimator= svc, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(n_splits=2), 
                          n_jobs=-1, 
                          scoring='precision', 
                          verbose=2).fit(X_train, y_train)


Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored